In [1]:
#This script opens the citation dataset and perform calculations in patent level
#Renato Kogeyama

#Miami, Jan 28, 2020
#Script to calculate citations received by a patent
#citation_id with less than 3 characters have no meaning
#they are causing problems moving forward
#so here I drop them

In [2]:
import pandas as pd
import numpy as np
import re
import glob
import dask.dataframe as dd

file_list=glob.glob("data/citation/*")
dfs = [pd.read_parquet(f, columns=['patent_id']).reset_index() for f in file_list]
dst = 'data/cit_made.parquet.gz'

#cited patents registers total citations received 

In [3]:
df=dfs[1]

In [4]:
df.head()

,citation_id,patent_id
0,4975840,7149720
1,6275369,9295828
2,5122180,10000965
3,8086206,8666347
4,7137006,7634803


In [5]:
df.groupby(['patent_id']).count()

,citation_id
patent_id,
10000005,1
10000007,6
10000010,1
10000011,2
10000023,2
...,...
RE48066,12
RE48067,3
RE48068,1


In [6]:
for i, df in enumerate(dfs):
    if i==0:
        output=df.groupby(['patent_id']).count() 
    else:
        output=pd.concat([output, df.groupby(['patent_id']).count()]) # following Nemet and Johson, this variable is relevant for citations made

In [7]:
# df=dfs.groupby(['patent_id']).count().iloc[:,0].reset_index() 

#Series, patent-level\n
#cited_patents.dropna(0, inplace=True) #Series, '0' implies that rows are dropped
#I should check this, because later I make citation back as index for merging purposes\n
# cited_patents=cited_patents.reset_index() #Dataframe")

output.rename(columns={'citation_id': 'cit_made'}, inplace=True)
# df['cit_made'].value_counts().sort_index()
# np.log(df['cit_made'].apply(lambda x: x+1)).hist()

In [8]:
# df['patent_id'].str.len().value_counts().sort_index()

# df=df[df['patent_id'].apply(lambda x: len(x)>4)]

In [9]:
output.head()

,cit_made
patent_id,
10000001,1
10000002,1
10000004,1
10000007,7
10000008,2


In [10]:
output=output.groupby(output.index).sum()

In [11]:
output.to_parquet(dst, compression='gzip')

In [12]:
output.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6993129 entries, 10000000 to RE48074
Data columns (total 1 columns):
 #   Column    Dtype
---  ------    -----
 0   cit_made  int64
dtypes: int64(1)
memory usage: 106.7+ MB
